In [8]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [9]:
# solving VFI

# placeholders, global objects

Toler = 0.5
State = np.arange(1,6)
# transition process 
F0=np.array([[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1],[0,0,0,0,1]])
F1=np.array([[1,0,0,0,0],[1,0,0,0,0],[1,0,0,0,0],[1,0,0,0,0],[1,0,0,0,0]])

global_args = (Toler, State, F0, F1)

# some parameter values
theta = -1
R = -3 



def VFI(theta, R, args = global_args):
    
    Toler, State, F0, F1 = global_args
    n = 0
    
    V1 = np.zeros((5,2))
    
    V0 = np.zeros((5,2))
    
    # Doing Value Function Iteration
    while np.abs(Toler)>0.0005:
        
        V1[:,0]= theta*State + 0.9*F0@np.log(np.sum(np.exp(V0),1))
        
        V1[:,1]= R + 0.9*F1@np.log(np.sum(np.exp(V0),1))
        
        Toler=np.max(np.max(np.abs(V1-V0)))
        V0=V1
        n +=1
        if n >= 10000:
            print('No Convergence')
            break
        
    return V0

data = pd.read_csv('data/rust.txt', sep=" ", header=None, names = ['State', 'Decision'])
data = data.groupby(['State','Decision'], as_index = False).size().values
K = data.shape[0]




In [10]:
data

array([[  1,   0, 172],
       [  1,   1,  24],
       [  2,   0, 122],
       [  2,   1,  74],
       [  3,   0,  59],
       [  3,   1, 166],
       [  4,   0,  28],
       [  4,   1, 178],
       [  5,   0,   9],
       [  5,   1, 168]])

In [11]:
def LL(params):

    theta, R = params[0], params[1]
    
    V = VFI(theta, R)
    L = 0
    
    for j in range(K):
        a, i, n = data[j][0],data[j][1],data[j][2]
        L += n*np.log( (np.exp(V[a-1,i]))/(np.exp(V[a-1,i]) + np.exp(V[a-1,1-i])) + 1e-30)
    
    return -L

In [12]:
# Q4.1
Res = VFI(-1,-3)

Res[1][1] - Res[1][0]

-0.7915423229149594

In [13]:
# Q4.2

# Res[3][1] + 1.5 - Res[3][0] - 1
# at those e the firm chooses to replace so we use replacement specific value

Res[3][1] - R

-1.3062606441574527

In [14]:
# Q5
res = minimize(LL, [-2,-3], method='Nelder-Mead', options={'disp': True})

res

Optimization terminated successfully.
         Current function value: 450.808896
         Iterations: 47
         Function evaluations: 89


 final_simplex: (array([[-1.01958224, -2.69594311],
       [-1.01962479, -2.6960386 ],
       [-1.01960052, -2.69594211]]), array([450.80889571, 450.80889576, 450.80889589]))
           fun: 450.80889570763276
       message: 'Optimization terminated successfully.'
          nfev: 89
           nit: 47
        status: 0
       success: True
             x: array([-1.01958224, -2.69594311])

In [7]:
# standard errors 
np.sqrt([0.0048, 0.034])

array([0.06928203, 0.18439089])